# Message Processing Semantics
## Actor Encapsulation

No direct access to actor behaviour is possible. Only possible by exchanging messages.

Messages can be send to known addresses(`ActorRef`)
- every actor knows its own address (self)
- creating an actor returns its address
- address can be sent within messages (e.g. `sender`)

It's not possible call methods on new created actors. Address can be sent within actors while sending messages.

Actors are completely independent agents of execution
- local execution, no notion of global syncrhonization. Every actor performs it's computation locally not shared directly with other actor (only by messages).
- all actors run fully concurrently
- message passing primitive is one-way communication. When aa actor sends a message it continues it's way.

One could say actors are completely isolated from each other. Actors are the most object oriented form of encapsulation. 

## Actor-Internal Evaluation Order
An actor is effectively single threaded.
- messages are recieved sequentially.
- behaviour change is effective before processing the next message.
- processing one message is the atomic unit of execution.

Everything an actor does for processing a single message cannot be interrupted or interleaved with processing of another message. This has the benifit of synchronized methods, but blocking is replaced by enqueing the messages.

## Bank Account - Revisited
It is good practice to define all actor messages (expected, returned, or replied) in a companion object

```scala
object BankAccount{
    // case class for depositing account
    case class Deposit(amount: BigInt){
        require(amount > 0)
    }
    //case class for withdrawing
    case class WithDraw(amount: BigInt){
        require(amount > 0)
    }
    // our actor respond to request
    case object done
    case object failed
}
class BankAccount extends Actor{
    import BankAccount._
    var balance = BigInt(0)
    def recieve = {
        case Deposit(amount) => balance += amount 
                                sender ! done
        case WithDraw(amount) if amount <= balance => balance -= amount sender ! done
        case _ => sender ! failed
    }
}
    
```

This actor is equivalent to bank account defined earlier included syncrhonization. Code here will be serialzed. One message entered cannot be intereferd with other. Other messages will be forced to wait.

## Actor Collabration
- picture actors as persons
- model activities as actors

Suppose we have two accounts for Alice and Bob. Now we want to transfer money. We could include code to transfer money. But bankaccount should not know how to transfer. We introduce another actor Tom, who will be model of activity to transfer money. Tom will first send message to Alice to withdraw the money and Alice will reply success. In case of failure Tom will abort transaction. Tom will deposit Bob. Bob will reply success. Tom can possibly shutdown and inform some one else.

